In [5]:
# Data manipulation and preprocessing
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
# Enables inline plotting in Jupyter/Colab notebooks
%matplotlib inline
np.set_printoptions(suppress=True)

# Scaling and model evaluation
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score, calinski_harabasz_score

# Machine learning models and tools
from sklearn.decomposition import PCA
from sklearn import cluster
from sklearn.cluster import DBSCAN, KMeans

##=== Pandas ==
import pandas as pd

# === Imports ===
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

In [6]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')  # Only needed if you're loading data from Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
original_data= pd.read_csv("/content/drive/MyDrive/Network Maintenace - With & without maintenace/Dataset/Treatment-encoded-dataset-2017-2020/df_encoded_2017_2020.csv")

In [10]:
original_data.shape

(6277, 27)

In [9]:
evaluate_data= pd.read_csv("/content/drive/MyDrive/Network Maintenace - With & without maintenace/Dataset/evaluated_test_set_with_predictions.csv")

In [11]:
evaluate_data.shape

(942, 28)

In [17]:
import pandas as pd

# Step 1: Identify common columns (excluding 'predicted_IRI')
columns_to_compare = [col for col in evaluate_data.columns if col in original_data.columns]

# Step 2: Subset and ensure column order matches
evaluate_subset = evaluate_data[columns_to_compare]
evaluate_subset = evaluate_subset[original_data.columns.intersection(columns_to_compare)]

# Step 3: Define row existence function
def row_exists(row):
    return ((original_data[columns_to_compare] == row.values).all(axis=1)).any()

# Step 4: Apply row-wise comparison
matches = evaluate_subset.apply(row_exists, axis=1)

# Step 5: Add match result to evaluate_data
evaluate_data['Exists_in_Original'] = matches

# Step 6: Print results
print(evaluate_data[['Exists_in_Original']].head(10))  # First 10 rows

# Step 7: Summary report
match_count = matches.sum()
total_count = len(evaluate_data)
match_percentage = 100 * match_count / total_count

print(f"\n✅ {match_count} out of {total_count} rows exist in original_data.")
print(f"✅ That’s {match_percentage:.2f}% matching rows.")

# Step 8: Optionally view matching/non-matching examples
print("\n🟩 First few matching rows:")
print(evaluate_data[evaluate_data['Exists_in_Original']].head())

print("\n🟥 First few non-matching rows:")
evaluate_data[~evaluate_data['Exists_in_Original']].head()


   Exists_in_Original
0               False
1               False
2               False
3               False
4               False
5               False
6               False
7               False
8               False
9               False

✅ 47 out of 942 rows exist in original_data.
✅ That’s 4.99% matching rows.

🟩 First few matching rows:
     AADT_mean_x  AADT_Single_Unit_mean_x  AADT_Combination_mean_x  \
29        1800.0                     39.0                     15.0   
49         609.0                     16.0                     56.0   
85       21000.0                    560.0                    640.0   
121        740.0                    140.0                    204.0   
129       2400.0                     66.0                     45.0   

     Future_AADT_mean_x  IRI_mean_x  Thickness_Rigid_mean_x  \
29               2245.0       121.3                     0.0   
49                731.0        76.4                     0.0   
85              22600.0        34.5         

AADT_mean_x  AADT_Single_Unit_mean_x  AADT_Combination_mean_x  \
0      21984.0                    807.0                    248.0   
1      72786.0                   4559.0                   2352.0   
2        435.0                     17.0                     15.0   
3      42749.0                   2309.0                    470.0   
4       2571.0                    154.0                     75.0   

   Future_AADT_mean_x  IRI_mean_x  Thickness_Rigid_mean_x  \
0             25941.0       120.0                     0.0   
1             94287.0        80.0                     9.0   
2               522.0        45.6                     0.0   
3             63523.0        35.3                     0.0   
4              2655.0       148.0                     0.0   

   Thickness_Flexible_mean_x  Base_Thickness_mean_x  F_System_mode  \
0                        9.0                   12.0            4.0   
1                        5.5                   16.0            2.0   
2                        4.0                    6.0            5.0   
3                        7.5                   42.0            2.0   
4                        6.0                   22.0            4.0   

   Surface_Type_mode  ...  Age_x  Urban_Type_rural  Urban_Type_small urban  \
0                2.0  ...   14.0               0.0                     0.0   
1                7.0  ...   54.0               0.0                     0.0   
2                2.0  ...   23.0               1.0                     0.0   
3                6.0  ...   34.0               1.0                     0.0   
4                6.0  ...   22.0               1.0                     0.0   

   Urban_Type_urban  Treatment_type_Resurfacing  Treatment_type_Thick Layers  \
0               1.0                         0.0                          0.0   
1               1.0                         0.0                          0.0   
2               0.0                         0.0                          0.0   
3               0.0                         0.0                          0.0   
4               0.0                         0.0                          0.0   

   Treatment_type_Thin Layers  Actual_IRI  Predicted_IRI  Exists_in_Original  
0                         1.0       132.0     135.322740               False  
1                         1.0        85.0      94.035720               False  
2                         1.0        56.5      55.058530               False  
3                         1.0        35.9      43.619724               False  
4                         1.0       197.0     174.213910               False  

[5 rows x 29 columns]